# 1. Import and Install Dependencies

In [ ]:
!pip install tensorflow==2.12.0 opencv-python mediapipe sklearn matplotlib

In [ ]:
pip install tensorflow

In [ ]:
pip install pandas

In [ ]:
pip install pyspark

In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [ ]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [ ]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [ ]:
def draw_styled_landmarks(image, results):
    # Draw face connections
    # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
    #                          mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
    #                          mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
    #                          ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

# 3. Extract Keypoint Values

In [ ]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    #face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 5. Collect Keypoint Values for Training and Testing

## ISL_CSLRT_Corpus video Files

In [ ]:
import pandas as pd

# Path to your CSV file (including the file name and extension)
csv_file_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\corpus_csv_files\ISL_CSLRT_Corpus details.csv"

# Read the CSV file into a pandas DataFrame
# Assuming UTF-8 encoding, but adjust if needed
df = pd.read_csv(csv_file_path)

# Define the root path using raw string literal
rootpath = 'C:/Users/ranjan.patra/OneDrive - Lingaro Sp. z o. o/DATA/IITJ/Course/Projects/ISL/'


In [ ]:
label_prev=""
#sample_num=0
df_k = pd.DataFrame(columns=['Label', 'sample_type','Sample_Number', 'Frame_Number', 'Keypoints'])
sample_type="Video"


for index, row in df.iterrows():
    #print("label_prev:",label_prev)
    #print(index)
    label = row["Sentences"]  # Get the "Word" value for the current row
    #print("label:",label)
    if label_prev!=label:
        sample_num=0
    else:
        sample_num=sample_num+1
    print("sample_num:",sample_num)
    label_prev=label
    video_path = rootpath + row["File location"]
    video_path = video_path.replace('/', '\\')
    print("video_path:",row["File location"])
    #video_path = r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_DICTIONARY\A\Alien_species.mp4"

    #for video_file in VIDEO_FILES:
    cap = cv2.VideoCapture(video_path)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        #print("Hi")
        #frame_number = 0
        #with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while cap.isOpened():
            ret, image = cap.read()
            if ret is None:
                print("Error reading frame. Exiting loop.")
                break
            elif not ret:
                print("End of video reached. Exiting loop.")
                break

     
            image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            image, results = mediapipe_detection(image, holistic)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            keypoints = extract_keypoints(results)
            frame_number=cap.get(cv2.CAP_PROP_POS_FRAMES)
            #print("Frame Num:",frame_number)
            #print("Frame Num:", frame_number, keypoints)
            

            #draw_styled_landmarks(image, results)
            


            #cv2.putText(image, 'Collecting frames for {}; Sample Number:{}; frame number:{}'.format(label,sample_num,frame_number), (15, 12),
                            #cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 255), 1, cv2.LINE_AA)


            #cv2.imshow('OpenCV Feed', image)

            #keypoints = extract_keypoints(results)
            new_data = pd.DataFrame({'Label': [label], 'sample_type': [sample_type],'Sample_Number': [sample_num], 'Frame_Number': [frame_number], 'Keypoints': [keypoints]})

            #new_data = pd.DataFrame({'Label': [label], 'Sample Number': [sample_num], 'Frame Number': [frame_number], 'Keypoints': [keypoints]})
            df_k = pd.concat([df_k, new_data], ignore_index=True)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

                frame_number += 1

        cap.release()
        print("Cap realease")
        cv2.destroyAllWindows()

df_k.to_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks")


In [ ]:
cap.release()
print("Cap realease")
cv2.destroyAllWindows()

In [ ]:



df_from_parquet = pd.read_parquet(r"C:\Users\ranjan.patra\OneDrive - Lingaro Sp. z o. o\DATA\IITJ\Course\Projects\ISL\ISL_CSLRT_Corpus\Mp_Data\LandMarks")

display(df_from_parquet)